# Sample dataset

In [1]:
import pandas as pd

In [2]:
schema = {
    'Features': [
        {
            'FeatureName': 'target',
            'FeatureType': 'float'
        },
        {
            'FeatureName': 'group_id_0',
            'FeatureType': 'int'
        },
        {
            'FeatureName': 'group_id_1',
            'FeatureType': 'int'
        },
        {
            'FeatureName': 'timestamp',
            'FeatureType': 'timestamp'
        }
    ]
}

In [3]:
df = pd.read_csv('data/target.csv')
df = df.rename(
    columns={
        'store': 'group_id_0',
        'item': 'group_id_1', 
        'date': 'timestamp', 
        'sales': 'target'
    }
)

In [4]:
df['target'] = df['target'].astype(float)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Dataset

In [5]:
import pyarrow.parquet as pq
import s3fs
from functools import reduce

In [6]:
class DatasetBridge:
    """Bridges buckets and dataset creation.
    """
    MINIO_ENDPOINT = 'http://minio:9000'
    ROOT_PATH = 'data/'

    def __init__(self, bucket_name, access_key, secret_key):
        self.bucket_name = bucket_name
        self.access_key = access_key
        self.secret_key = secret_key        

    def get_parquet_dataset(self, base_dir):
        s3_path = self._make_s3_root_path(base_dir)
        fs = self._get_s3_filesystem()
        parquet_dataset = pq.ParquetDataset(s3_path, filesystem=fs)
        return parquet_dataset

    def _make_s3_root_path(self, *args):
        path = ("s3://" +
                # self.ROOT_PATH +
                self.bucket_name +
                '/' +
                '/'.join(args))
        return path

    def _get_s3_filesystem(self):
        client_kwargs = {
            'endpoint_url': self.MINIO_ENDPOINT,
            'aws_access_key_id': self.access_key,
            'aws_secret_access_key': self.secret_key,
            'verify': False
        }
        fs = s3fs.S3FileSystem(anon=False, use_ssl=False,
                               client_kwargs=client_kwargs)

        return fs
    
    
class Dataset:
    """Interface for parquet datasets.
    """
    def __init__(self, parquet_ds):
        self.parquet_ds = parquet_ds
        
    def get_pandas_df(self):
        return self.parquet_ds.read_pandas().to_pandas()
    
    def get_group_ids(self):
        arrow_schema = self.get_arrow_schema()
        return [x for x in arrow_schema.names if x.startswith('group_id')]
        
    def get_arrow_schema(self):
        return self.parquet_ds.schema.to_arrow_schema()
    
    def get_names(self):
        arrow_schema = self.get_arrow_schema()
        return arrow_schema.names
    
    def merge(self, parquet_ds):
        left_df = self.get_pandas_df()
        right_df = parquet_ds.get_pandas_df()
        group_ids = self.get_group_ids()
        merged_df = pd.merge(
            left=left_df,
            right=right_df,
            on=group_ids + ['timestamp']
        )
        return merged_df
    
    
class DatasetsCollector:
    """Base class for dataset collectors.
    """
    def __init__(self, **kwargs):
        self._validate_kwargs(**kwargs)
        self.kwargs = kwargs
        vars(self).update(self.kwargs)
        
        
    def get_dataset_by_name(self, name):
        if name not in self.kwargs:
            raise ValueError('Unkown dataset name {}'.format(name))
        return self.kwargs[name]
        
    def get_names(self, include_pk=True):
        names = {
            k: v.get_names() 
            if v is not None 
            else [] for k, v in self.kwargs.items()
        }
        if not include_pk:
            group_ids = self.get_group_ids()
            pk = group_ids + ['timestamp']
            names_without_pk = {
                k: [x for x in v if x not in pk] 
                for k, v in names.items()
            }
            return names_without_pk
        return names
        
    def _get_datasets(self):
        return list(self.kwargs.values())
        
        
    def get_group_ids(self, validate=True):
        datasets = self._get_datasets()
        all_group_ids = [ds.get_group_ids() for ds in datasets if ds is not None]
         
        # Uniqueness for list of lists.
        unique_group_ids = [list(x) for x in set(tuple(x) for x in all_group_ids)]
        
        if validate:
            if len(unique_group_ids) > 1:
                raise
            return unique_group_ids[0]
        return unique_group_ids
    
    def merge(self):
        datasets = self._get_datasets()
        dfs = [ds.get_pandas_df() for ds in datasets if ds is not None]
        
        pk = self.get_group_ids() + ['timestamp']
        df_merge = reduce(lambda left, right: pd.merge(left, right, on=pk), dfs)
        return df_merge
        
        
        
    def _validate_kwargs(self, **kwargs):
        for key, dataset in kwargs.items():
            if dataset is not None:
                if not isinstance(dataset, Dataset):
                    raise TypeError(
                        'All parameters must be of type Dataset. ' 
                        'Instead, kwarg {} received type {}'.format(key, type(dataset).__name__)
                    )
                    
                    
class TimeSeriesCollector(DatasetsCollector):
    def __init__(
        self, 
        target, 
        time_varying_known_reals=None, 
        time_varying_unknown_reals=None, 
        static_categoricals=None
    ):
        super().__init__(
            target=target, 
            time_varying_known_reals=time_varying_known_reals, 
            time_varying_unknown_reals=time_varying_unknown_reals, 
            static_categoricals=static_categoricals
        )
        
        

# Predictor

In [7]:
from pydantic import BaseModel

In [8]:
class Predictor(BaseModel):
    name: str
    algorithm: str
    forecast_horizon = int
    perform_hpo: bool = False
    bucket_name: str

# Estimator

In [9]:
from mooncake.nn import SeqToSeq, TemporalFusionTransformer as TFT
from mooncake.helper import common_callbacks
from torch.optim.lr_scheduler import OneCycleLR


class EstimatorCreator:
    
    ESTIMATORS = {
    'seq2seq': SeqToSeq,
    'tft': TFT
    }
    
    def __init__(self, predictor, dataset_collector):
        self.predictor = predictor
        self.dataset_collector = dataset_collector

    def create_estimator(self):
        cls = self._get_estimator_class()
        estimator_args = self._get_estimator_args()
        return cls(**estimator_args)

    def _get_estimator_class(self):
        return self.ESTIMATORS[self.predictor.algorithm]

    def _get_estimator_args(self):
        args_creator = EstimatorArgsCreator(self.predictor, self.dataset_collector)
        return args_creator.get_estimator_args()


class EstimatorArgsCreator:
    def __init__(self, predictor, dataset_collector):
        self.predictor = predictor
        self.dataset_collector = dataset_collector

    def get_estimator_args(self):
        time_segmentation = self.dataset_collector.get_names(include_pk=False)
        group_ids = self.dataset_collector.get_group_ids()
        max_prediction_length = self.predictor.forecast_horizon
        time_idx = 'time_idx'
        
        # Callbacks.
        lr_scheduler = dict(
        policy=OneCycleLR, 
        step_every='batch', 
        max_lr=1e-3, 
        steps_per_epoch='iterations', 
        epochs='max_epochs'
        )
        callbacks = common_callbacks(lr_scheduler, gradient_clipping=True)
            
        return {
            'group_ids': group_ids,
            'max_prediction_length': max_prediction_length,
            'time_idx': time_idx,
            'max_encoder_length': 20,
            'callbacks': callbacks,
            **time_segmentation
        }

# Preprocessor

In [10]:
from mooncake.preprocessing import (
    GroupTransformer,
    DataframeColumnTransformer,
    CyclicalDates
)
from mooncake.helper import column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


class PreprocessorCreator:
    def __init__(self, group_ids, scaler=MinMaxScaler(),
                 encoder=OneHotEncoder(), cyclical_dates=True,
                 time_idx='time_idx'):
        self.group_ids = group_ids
        self.scaler = scaler
        self.encoder = encoder
        self.cyclical_dates = cyclical_dates
        self.time_idx = time_idx

    def create_preprocessor(self):
        group_trans = self._create_group_transformations()
        outer_trans = self._create_outer_transformations()

        steps = [
            ("groups", GroupTransformer(group_trans, self.group_ids)),
            ("outer", DataframeColumnTransformer(outer_trans))
        ]
        return Pipeline(steps=steps)

    def _create_group_transformations(self):
        """Transformers defined here will act group by group.
        That is, each group will be fitted to its own set of transformers.
        """
        selector = column_selector(
            dtype_include=['float', 'int'],
            pattern_exclude=self.group_ids + [self.time_idx]
        )
        group_transformations = [
            ('cont', self.scaler, selector),
        ]
        return group_transformations

    def _create_outer_transformations(self):
        timestamp_column = 'timestamp'
        selector = column_selector(
            dtype_include=['object'],
            pattern_exclude=self.group_ids
        )
        outer_transformations = [
            #('cat', self.encoder, selector),
        ]
        if self.cyclical_dates:
            outer_transformations.append(
                ('dates', CyclicalDates(), timestamp_column)
            )
        return outer_transformations


# Putting it all together 

In [11]:
# Data bridge between minio and mlpi.
bridge = DatasetBridge(bucket_name='sample', access_key='oxxo', secret_key='password')

# Create ``dataset`` object which is a easy-to-use parquet dataset interface.
parquet_dataset = bridge.get_parquet_dataset('target')
dataset = Dataset(parquet_dataset)

# Collect all datasets in a `DatasetCollector` object
dataset_collector = TimeSeriesCollector(
    target=dataset, 
    time_varying_known_reals=None, 
    time_varying_unknown_reals=None, 
    static_categoricals=None
)

In [12]:
# Predictor object (request json)

json_request = {
    'name': 'sample',
    'algorithm': 'seq2seq',
    'forecast_horizon': 10,
    'perform_hpo': False,
    'bucket_name': 'sample',
}

predictor = Predictor(**json_request)

In [13]:
# Create estimator

estimator = EstimatorCreator(predictor, dataset_collector).create_estimator()

In [15]:
# Preprocess

group_ids = dataset_collector.get_group_ids()
X = dataset_collector.merge()
X = X.astype({g: str for g in group_ids})
group_ids = dataset_collector.get_group_ids()
prepocessor_creator = PreprocessorCreator(group_ids)
preprocessor = prepocessor_creator.create_preprocessor()

In [16]:
preprocessor.fit_transform(X)

,day_sine,day_cos,month_sine,month_cos,target,group_id_0,group_id_1
0,0.201299,0.394356,0.5,0.5,0.105960,10,13
1,0.571268,0.724793,0.5,0.5,0.178808,10,13
2,0.848644,0.937752,0.5,0.5,0.039735,10,13
3,0.988468,0.998717,0.5,0.5,0.158940,10,13
4,0.968077,0.897805,0.5,0.5,0.165563,10,13
...,...,...,...,...,...,...,...
91295,-0.250653,-0.050649,1.0,1.0,0.220238,9,28
91296,0.151428,0.347305,1.0,1.0,0.303571,9,28
91297,0.528964,0.688967,1.0,1.0,0.404762,9,28
91298,0.820763,0.918958,1.0,1.0,0.380952,9,28
